In [1]:
%matplotlib inline

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import nengo
from nengo.utils.filter_design import cont2discrete
import numpy as np
import tensorflow as tf
from dv import LegacyAedatFile

import nengo_dl

from IPython.display import clear_output

## Gesture Dataset Environment

In [2]:
def load_gesture(name, gesture):
    with open("../DVS_Gesture_Dataset/DvsGesture/"+name[:-7]+"_labels.csv", "r") as l:
        for line in l:
            labels = line.split(",")
            if labels[0] == str(gesture):
                start = int(labels[1])
                end = int(labels[2])
    
    events = []
    with LegacyAedatFile("../DVS_Gesture_Dataset/DvsGesture/"+name[:-1]) as f:
        for event in f:
            if event.timestamp >= start and event.timestamp <= end:
                events.append([event.x/128, event.y/128])
    return np.array(events)

def trim_gesture(gesture, gesture_size):
    if gesture.shape[0] > gesture_size:
        gesture = gesture[0:gesture_size]
    else:
        temp = np.zeros(gesture_size, 3)
        temp[:gesture.shape[0]] = gesture
        gesture = temp
    return gesture

gesture_size = 50000

gestures = []
labels = []
count = 1
with open("../DVS_Gesture_Dataset/DvsGesture/trials_to_train.txt") as file:
    for name in file:
        for label in range(1, 3):
            try:
                gestures.append(trim_gesture(load_gesture(name, label), gesture_size))
                labels.append(label-1)
                print("loaded", name, count)
                count += 1
            except:
                print("failed to load", name)

gestures = np.array(gestures)
temp = np.zeros((len(labels), 1, 1))
temp[:, 0, 0] = np.array(labels)
labels = temp

loaded user01_fluorescent.aedat
 1
loaded user01_fluorescent.aedat
 2
loaded user01_fluorescent_led.aedat
 3
loaded user01_fluorescent_led.aedat
 4
loaded user01_lab.aedat
 5
loaded user01_lab.aedat
 6
loaded user01_led.aedat
 7
loaded user01_led.aedat
 8
loaded user01_natural.aedat
 9
loaded user01_natural.aedat
 10
loaded user02_fluorescent.aedat
 11
loaded user02_fluorescent.aedat
 12
loaded user02_fluorescent_led.aedat
 13
loaded user02_fluorescent_led.aedat
 14
failed to load user02_lab.aedat

loaded user02_lab.aedat
 15
loaded user02_led.aedat
 16
loaded user02_led.aedat
 17
loaded user02_natural.aedat
 18
loaded user02_natural.aedat
 19
loaded user03_fluorescent.aedat
 20
loaded user03_fluorescent.aedat
 21
loaded user03_fluorescent_led.aedat
 22
loaded user03_fluorescent_led.aedat
 23
loaded user03_led.aedat
 24
loaded user03_led.aedat
 25
loaded user03_natural.aedat
 26
loaded user03_natural.aedat
 27
loaded user04_fluorescent.aedat
 28
loaded user04_fluorescent.aedat
 29
load

loaded user29_natural.aedat
 242


In [4]:
np.save("gestures", gestures)
np.save("labels", labels)

In [2]:
gestures = np.load("gestures.npy")
labels = np.load("labels.npy")

## Legendre Memory Unit Cell Definition

In [3]:
class LMUCell(nengo.Network):
    def __init__(self, units, order, theta, input_d, **kwargs):
        super().__init__(**kwargs)

        # compute the A and B matrices according to the LMU's mathematical derivation
        # (see the paper for details)
        Q = np.arange(order, dtype=np.float64)
        R = (2 * Q + 1)[:, None] / theta
        j, i = np.meshgrid(Q, Q)

        A = np.where(i < j, -1, (-1.0) ** (i - j + 1)) * R
        B = (-1.0) ** Q[:, None] * R
        C = np.ones((1, order))
        D = np.zeros((1,))

        A, B, _, _, _ = cont2discrete((A, B, C, D), dt=1.0, method="zoh")

        with self:
            nengo_dl.configure_settings(trainable=None)

            # create objects corresponding to the x/u/m/h variables in the above diagram
            self.x = nengo.Node(size_in=input_d)
            self.u = nengo.Node(size_in=1)
            self.m = nengo.Node(size_in=order)
            self.h = nengo_dl.TensorNode(tf.nn.tanh, shape_in=(units,), pass_time=False)

            # compute u_t from the above diagram.
            # note that setting synapse=0 (versus synapse=None) adds a one-timestep
            # delay, so we can think of any connections with synapse=0 as representing
            # value_{t-1}
            nengo.Connection(
                self.x, self.u, transform=np.ones((1, input_d)), synapse=None
            )
            nengo.Connection(self.h, self.u, transform=np.zeros((1, units)), synapse=0)
            nengo.Connection(self.m, self.u, transform=np.zeros((1, order)), synapse=0)

            # compute m_t
            # in this implementation we'll make A and B non-trainable, but they
            # could also be optimized in the same way as the other parameters
            conn_A = nengo.Connection(self.m, self.m, transform=A, synapse=0)
            self.config[conn_A].trainable = False
            conn_B = nengo.Connection(self.u, self.m, transform=B, synapse=None)
            self.config[conn_B].trainable = False

            # compute h_t
            nengo.Connection(
                self.x, self.h, transform=np.zeros((units, input_d)), synapse=None
            )
            nengo.Connection(
                self.h, self.h, transform=np.zeros((units, units)), synapse=0
            )
            nengo.Connection(
                self.m,
                self.h,
                transform=nengo_dl.dists.Glorot(distribution="normal"),
                synapse=None,
            )

## Network Definition

In [4]:
tau_slow = 0.01
tau_fast = None
discount = 0.95

class Sizes:
    edge_dvs = 128
    edge_sptc = 32
    pop_dvs = edge_dvs ** 2
    pop_sptc = edge_sptc ** 2
    actions = 2

with nengo.Network() as net:
    nengo_dl.configure_settings(
        trainable=None,
        stateful=False,
        keep_history=False,
    )
    
    # input node
    inp = nengo.Node(np.zeros(gestures.shape[-1]))
    
    # LMU cell
    lmu = LMUCell(
        units=212,
        order=256,
        theta=gestures.shape[1], # number of events per gesture
        input_d=gestures.shape[-1], # number of dimension per event (3)
    )
    conn = nengo.Connection(inp, lmu.x, synapse=None)
    net.config[conn].trainable = False
    
    # dense linear readout
    out = nengo.Node(size_in=2)
    nengo.Connection(lmu.h, out, transform=nengo_dl.dists.Glorot(), synapse=None)

    # record output. note that we set keep_history=False above, so this will
    # only record the output on the last timestep (which is all we need
    # on this task)
    p = nengo.Probe(out)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(gestures, labels, test_size=0.33, random_state=42)

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=8, unroll_simulation=8) as sim:
    sim.compile(
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.optimizers.Adam(),
        metrics=["accuracy"],
    )

    sim.load_params("./lmu_params")
    sim.fit(X_train, y_train, epochs=2)
    sim.save_params("./lmu_params")

    print(
        "Final test accuracy: %.2f%%"
        % (sim.evaluate(X_test, y_test, verbose=0)["probe_accuracy"] * 100)
    )

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/2
|#             Constructing graph: build stage (1%)              | ETA: 0:00:03

/home/thomas/anaconda3/envs/rl-snn/lib/python3.8/site-packages/nengo_dl/simulator.py:1929: UserWarning: Number of elements in input data (162) is not evenly divisible by Simulator.minibatch_size (8); input data will be truncated.
  warnings.warn(


10/20 [==============>...............] - ETA: 33:02 - loss: 0.6615 - probe_loss: 0.6615 - probe_accuracy: 0.5375

In [ ]:
with nengo_dl.Simulator(net, minibatch_size=8, unroll_simulation=8) as sim:
    sim.save_params("./lmu_params")